In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import pandas as pd
import glob

In [45]:
# load gfdl_dataset with low emission scenario")
ds=xr.open_dataset("/A04/so_data/CNN_data/Proposed_cmip6/UKESM1-0-LL_ssp245_r1i1p1f1_gr_201501-210012_fixed.nc")
ds

<xarray.Dataset> Size: 2GB
Dimensions:    (time: 1032, latitude: 180, longitude: 360)
Coordinates:
  * latitude   (latitude) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * longitude  (longitude) float64 3kB -180.0 -179.0 -178.0 ... 178.0 179.0
    mask       (latitude, longitude) float64 518kB ...
  * time       (time) datetime64[ns] 8kB 2015-01-01 2015-02-01 ... 2100-12-01
Data variables:
    chlos      (time, latitude, longitude) float32 267MB ...
    mlotst     (time, latitude, longitude) float32 267MB ...
    rsds       (time, latitude, longitude) float32 267MB ...
    siconc     (time, latitude, longitude) float32 267MB ...
    tos        (time, latitude, longitude) float32 267MB ...
    uas        (time, latitude, longitude) float32 267MB ...
    vas        (time, latitude, longitude) float32 267MB ...
    zos        (time, latitude, longitude) float32 267MB ...
Attributes: (12/49)
    CDI:                    Climate Data Interface version 1.9.8 (https://mpi...
    history:                Fri Jul 18 21:20:52 2025: cdo remapbil,r360x180 c...
    source:                 UKESM1.0-LL (2018): \naerosol: UKCA-GLOMAP-mode\n...
    institution:            Met Office Hadley Centre, Fitzroy Road, Exeter, D...
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            ScenarioMIP
    ...                     ...
    variable_id:            chlos
    variable_name:          chlos
    variant_label:          r3i1p1f2
    license:                CMIP6 model data produced by the Met Office Hadle...
    cmor_version:           3.4.0
    CDO:                    Climate Data Operators version 1.9.8 (https://mpi...

In [46]:
ds_bathy = xr.open_dataset('/A04/so_data/DATA/ETOPO1_Ice_g_gmt4.grd', engine='netcdf4')
res = ds_bathy.z.interp(x=ds.longitude, y=ds.latitude,method = 'linear')
ds = ds.assign(variables={"bathymetry": (('latitude','longitude'), res.data)})
ds = ds.where(ds.bathymetry < -200)

In [47]:
time_range = pd.date_range(start='2015-01-01', end='2100-12-31', freq='ME')
ds = ds.assign_coords(time=time_range)
ds['time'] = pd.to_datetime(ds['time']).to_period('M').to_timestamp()
ds

<xarray.Dataset> Size: 2GB
Dimensions:     (time: 1032, latitude: 180, longitude: 360)
Coordinates:
  * latitude    (latitude) float64 1kB -89.5 -88.5 -87.5 ... 87.5 88.5 89.5
  * longitude   (longitude) float64 3kB -180.0 -179.0 -178.0 ... 178.0 179.0
    mask        (latitude, longitude) float64 518kB 0.0 0.0 0.0 ... 1.0 1.0 1.0
  * time        (time) datetime64[ns] 8kB 2015-01-01 2015-02-01 ... 2100-12-01
Data variables:
    chlos       (time, latitude, longitude) float32 267MB nan nan ... 2.085e-08
    mlotst      (time, latitude, longitude) float32 267MB nan nan ... 237.9
    rsds        (time, latitude, longitude) float32 267MB nan nan ... 0.0 0.0
    siconc      (time, latitude, longitude) float32 267MB nan nan ... 0.0 0.0
    tos         (time, latitude, longitude) float32 267MB nan nan ... 1.979
    uas         (time, latitude, longitude) float32 267MB nan nan ... -2.927
    vas         (time, latitude, longitude) float32 267MB nan nan ... 3.838
    zos         (time, latitude, longitude) float32 267MB nan nan ... -0.9065
    bathymetry  (latitude, longitude) float64 518kB nan nan ... -4.307e+03
Attributes: (12/49)
    CDI:                    Climate Data Interface version 1.9.8 (https://mpi...
    history:                Fri Jul 18 21:20:52 2025: cdo remapbil,r360x180 c...
    source:                 UKESM1.0-LL (2018): \naerosol: UKCA-GLOMAP-mode\n...
    institution:            Met Office Hadley Centre, Fitzroy Road, Exeter, D...
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            ScenarioMIP
    ...                     ...
    variable_id:            chlos
    variable_name:          chlos
    variant_label:          r3i1p1f2
    license:                CMIP6 model data produced by the Met Office Hadle...
    cmor_version:           3.4.0
    CDO:                    Climate Data Operators version 1.9.8 (https://mpi...

In [48]:
weights = np.cos(np.deg2rad(ds.latitude))
weights.name = "weights"
dsw = ds['zos'].weighted(weights)
zos_mean = dsw.mean(dim = ("longitude", "latitude"))
ds = ds.assign(variables={"zos_mean": (('time'), zos_mean.data)}) 

#Remove the global weighted mean by timestep
attrs = ds['zos'].attrs
zos_new = (ds['zos']-ds['zos_mean'])
ds['zos'] = zos_new
ds['zos'].attrs = attrs
ds = ds.drop_vars('zos_mean')
ds

<xarray.Dataset> Size: 2GB
Dimensions:     (time: 1032, latitude: 180, longitude: 360)
Coordinates:
  * latitude    (latitude) float64 1kB -89.5 -88.5 -87.5 ... 87.5 88.5 89.5
  * longitude   (longitude) float64 3kB -180.0 -179.0 -178.0 ... 178.0 179.0
    mask        (latitude, longitude) float64 518kB 0.0 0.0 0.0 ... 1.0 1.0 1.0
  * time        (time) datetime64[ns] 8kB 2015-01-01 2015-02-01 ... 2100-12-01
Data variables:
    chlos       (time, latitude, longitude) float32 267MB nan nan ... 2.085e-08
    mlotst      (time, latitude, longitude) float32 267MB nan nan ... 237.9
    rsds        (time, latitude, longitude) float32 267MB nan nan ... 0.0 0.0
    siconc      (time, latitude, longitude) float32 267MB nan nan ... 0.0 0.0
    tos         (time, latitude, longitude) float32 267MB nan nan ... 1.979
    uas         (time, latitude, longitude) float32 267MB nan nan ... -2.927
    vas         (time, latitude, longitude) float32 267MB nan nan ... 3.838
    zos         (time, latitude, longitude) float64 535MB nan nan ... -0.9065
    bathymetry  (latitude, longitude) float64 518kB nan nan ... -4.307e+03
Attributes: (12/49)
    CDI:                    Climate Data Interface version 1.9.8 (https://mpi...
    history:                Fri Jul 18 21:20:52 2025: cdo remapbil,r360x180 c...
    source:                 UKESM1.0-LL (2018): \naerosol: UKCA-GLOMAP-mode\n...
    institution:            Met Office Hadley Centre, Fitzroy Road, Exeter, D...
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            ScenarioMIP
    ...                     ...
    variable_id:            chlos
    variable_name:          chlos
    variant_label:          r3i1p1f2
    license:                CMIP6 model data produced by the Met Office Hadle...
    cmor_version:           3.4.0
    CDO:                    Climate Data Operators version 1.9.8 (https://mpi...

In [49]:
# Save the processed dataset
ds.to_netcdf("/A04/so_data/CNN_data/global_processed/UKESM1-0-LL_ssp245_r1i1p1f1_gr_201501-210012_processed.nc")